In [140]:
%cd /home/cloudera/Dropbox/DataScienceProgram/Day2

/home/cloudera/Dropbox/DataScienceProgram/Day2


In [143]:
!mkdir HWDay2

In [146]:
%cd HWDay2/

/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2


In [147]:
!pwd

/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2


In [149]:
#3. HW2.1 
#Change the mapper.py/reducer.py combination from the the above wordcount example so that you get 
#the longest word present in the Alice Book (from HW1).

In [156]:
!curl 'http://www.gutenberg.org/cache/epub/11/pg11.txt' -o alicesTExtFilename.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  163k  100  163k    0     0  41487      0  0:00:04  0:00:04 --:--:-- 45447


In [157]:
!head alicesTExtFilename.txt

In [197]:
%%writefile mapper1.py
#!/usr/bin/env python

import sys
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

wordLen=0
wordLong=None

for line in sys.stdin:
    #This removes the whitespaces
    line = line.strip()
    #getting words from line
    words=line.split()
    #getting words and their counts
    for word in words:
        if len(word)>wordLen:
            wordLen=len(word)
            wordLong=word
              
    print ('%05d\t%s') % (wordLen, wordLong)

Overwriting mapper1.py


In [198]:
!chmod a+x reducer1.py
!chmod a+x mapper1.py

In [202]:
!echo "You are my world bro" | mapper1.py 

/bin/sh: mapper1.py: command not found


In [203]:
%%writefile reducer1.py
#!/usr/bin/env python

import sys

curmax_wordLen = 0
curmax_wordLong = None
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")

for line in sys.stdin:
    line = line.split()
    wordLen, wordLong = line.split('\t', 1)
    
    if wordLen>curmax_wordLen:
        curmax_wordLen=wordLen
        curmax_wordLong=wordLong
            
print ('%s\t%s') % (curmax_wordLong, curmax_wordLen)

Overwriting reducer1.py


In [204]:
!hdfs dfs -rm alicesTExtFilename.txt 
!hdfs dfs -copyFromLocal alicesTExtFilename.txt 
!hdfs dfs -rm -r curmax_wordLen-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
dataDir = "/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/"

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
   -file "/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapper1.py" \
   -mapper /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapper1.py \
   -file "/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer1.py" \
   -reducer /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer1.py \
   -combiner /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer1.py \
   -input alicesTExtFilename.txt \
   -output curmax_wordLen-output  \
   -numReduceTasks 3
   #--D mapreduce.job.reduces=2  depecated
#-input historical_tours.txt  file on Hadoop


#output directory on Hadoop 

Deleted alicesTExtFilename.txt
rm: `curmax_wordLen-output': No such file or directory
16/08/31 11:13:13 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapper1.py, /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer1.py] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob6368756509374150966.jar tmpDir=null
16/08/31 11:13:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 11:13:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 11:13:15 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/31 11:13:15 INFO mapreduce.JobSubmitter: number of splits:2
16/08/31 11:13:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472467579313_0017
16/08/31 11:13:15 INFO impl.YarnClientImpl: Submitted application application_1472467579313_0017
16/08/31 11:13:15 INFO ma

In [51]:
# 3. HW2.2
Change the mapper.py/reducer.py combination so that you get only the number of words starting with an uppercase letter, 
and the number of words starting with a lowercase letter. 

In [207]:
%%writefile mapperupper.py
#!/usr/bin/env python

import sys
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

Upper_Case = 0
Lower_Case = 0
for line in sys.stdin:
    for line in sys.stdin:
    for word in line.split():
        if word[0].isupper:
        print '%s\t%d' % (word, 1)

Overwriting mapperupper.py


In [208]:
%%writefile mapperlower.py
#!/usr/bin/env python

import sys
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

Upper_Case = 0
Lower_Case = 0
for line in sys.stdin:
    for line in sys.stdin:
    for word in line.split():
        if word[0].islower:
        print '%s\t%d' % (word, 1)

Overwriting mapperlower.py


In [209]:
%%writefile reducer2.py
#!/usr/bin/env python

import sys

cur_key = None
cur_count = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)

Writing reducer2.py


In [211]:
!hdfs dfs -rm alicesTExtFilename.txt 
!hdfs dfs -copyFromLocal alicesTExtFilename.txt 
!hdfs dfs -rm -r cur_key-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
dataDir = "/home/cloudera/Dropbox/DataScienceProgram/Day2"

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
   -file "/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapperupper.py" \
   -mapper /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapperupper.py \
   -file "/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer2.py" \
   -reducer /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer2.py \
   -combiner /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer2.py \
   -input alicesTExtFilename.txt \
   -output cur_key-output  \
   -numReduceTasks 3
   #--D mapreduce.job.reduces=2  depecated
#-input historical_tours.txt  file on Hadoop


#output directory on Hadoop 

Deleted alicesTExtFilename.txt
Deleted cur_key-output
16/08/31 11:21:52 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapperupper.py, /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer2.py] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob2882959984729867381.jar tmpDir=null
16/08/31 11:21:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 11:21:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 11:21:53 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/31 11:21:53 INFO mapreduce.JobSubmitter: number of splits:2
16/08/31 11:21:54 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472467579313_0019
16/08/31 11:21:54 INFO impl.YarnClientImpl: Submitted application application_1472467579313_0019
16/08/31 11:21:54 INFO mapreduce.Job: The url to trac

In [212]:
!hdfs dfs -rm alicesTExtFilename.txt 
!hdfs dfs -copyFromLocal alicesTExtFilename.txt 
!hdfs dfs -rm -r cur_key-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
dataDir = "/home/cloudera/Dropbox/DataScienceProgram/Day2"

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
   -file "/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapperlower.py" \
   -mapper /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapperlower.py \
   -file "/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer2.py" \
   -reducer /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer2.py \
   -combiner /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer2.py \
   -input alicesTExtFilename.txt \
   -output cur_key-output  \
   -numReduceTasks 3
   #--D mapreduce.job.reduces=2  depecated
#-input historical_tours.txt  file on Hadoop


#output directory on Hadoop 

Deleted alicesTExtFilename.txt
Deleted cur_key-output
16/08/31 11:23:05 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/mapperlower.py, /home/cloudera/Dropbox/DataScienceProgram/Day2/HWDay2/reducer2.py] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob7673215411723592499.jar tmpDir=null
16/08/31 11:23:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 11:23:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/31 11:23:07 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/31 11:23:07 INFO mapreduce.JobSubmitter: number of splits:2
16/08/31 11:23:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472467579313_0020
16/08/31 11:23:07 INFO impl.YarnClientImpl: Submitted application application_1472467579313_0020
16/08/31 11:23:07 INFO mapreduce.Job: The url to trac

In [71]:
3. HW2.3

The data file at http://www.metoffice.gov.uk/climate/uk/stationdata/heathrowdata.txt (mirrored here) is part of a 
set of text files cataloguing temperatures collected in different cities of the UK. The one for the URL given is for 
Heathrow (London Airport). Typically each city is held in one data file. You can access the whole set from here. 
Write a MapReduce application with a mapper.py and reduce.py that uses only the text file for Heathrow as input and 
that outputs the lowest temperature ever recorded and the year this event happened.

SyntaxError: invalid syntax (<ipython-input-71-3ee0e3edc958>, line 1)

In [72]:
!curl 'http://www.metoffice.gov.uk/climate/uk/stationdata/heathrowdata.txt' -o heathrowdata.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   292  100   292    0     0    169      0  0:00:01  0:00:01 --:--:--   215


In [74]:
!head heathrowdata.txt

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="http://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/heathrowdata.txt">here</a>.</p>
</body></html>
